# GNN-based Sequential Recommendation System

This notebook implements a Graph Neural Network (GNN) for sequential recommendation. It covers data preprocessing, graph construction, and model training, followed by evaluation using metrics like ROC-AUC. The purpose of this notebook is to demonstrate the workflow for developing and testing a GNN-based recommendation system.


In [ ]:
## Importing Required Libraries

Below are the essential libraries used for implementing the model, preprocessing, and evaluation functions:
- `torch`: Deep learning framework for building neural networks.
- `tqdm`: Used for progress bars during model training and evaluation.
- `sklearn`: Provides utility functions for evaluation metrics.


In [1]:
import pandas as pd
import torch
# Load and preprocess the dataset
train_data = pd.read_csv(r"/Users/vishnusai/Downloads/archive-4/train_csv_new.csv")


In [2]:
from recommenders.datasets.split_utils import filter_k_core
train_data = train_data[["user_id", "product_id", "timestamp"]]
train_data.rename(columns = {'user_id':'userID','product_id':'itemID','timestamp':'time'}, inplace = True)
train_data = filter_k_core(train_data, 10)

(1211692, 3)

In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

train_data['userID'] = label_encoder.fit_transform(train_data['userID'])
train_data['itemID'] = label_encoder.fit_transform(train_data['itemID'])

train_sequences = train_data.groupby('userID')['itemID'].apply(list).reset_index()

train_sequences['sequences_length'] = train_sequences['itemID'].apply(len)
average_length = train_sequences['sequences_length'].mean()
print(f"The average length of the sequences is: {average_length}")


The average length of the sequences is: 17.86813737778892


In [4]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Define the GNNRecommender class with GRU instead of LSTM
class GNNRecommender(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super(GNNRecommender, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.gcn1 = GCNConv(embedding_dim, embedding_dim)
        self.gcn2 = GCNConv(embedding_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, embedding_dim, batch_first=True) 
        self.fc = nn.Linear(embedding_dim, num_items)

    def forward(self, edge_index, sequences):
        user_emb = self.user_embedding.weight
        item_emb = self.item_embedding.weight
        x = torch.cat([user_emb, item_emb], dim=0)
        x = F.relu(self.gcn1(x, edge_index))
        x = F.dropout(x, p=0.3, training=self.training)
        x = F.relu(self.gcn2(x, edge_index))
        seq_emb = self.item_embedding(sequences)
        lstm_out, _ = self.lstm(seq_emb)
        out = self.fc(lstm_out) 
        return out



## Graph Construction

The following function builds a graph for the input sequences. It converts sequences of interactions into a format compatible with PyTorch geometric models.


In [5]:
import torch
import numpy as np

# Function to build graph from sequences
def build_graph(sequences):
    users = sequences['userID'].values
    items = sequences['itemID'].apply(lambda x: x[:-1]).values
    next_items = sequences['itemID'].apply(lambda x: x[1:]).values

    # Create user-item pairs (edges)
    user_item_pairs = [(u, i) for u, item_seq in zip(users, items) for i in item_seq]
    
    # Create item-item pairs (edges for sequential recommendations)
    item_item_pairs = [(i, j) for item_seq in zip(items, next_items) for i, j in zip(item_seq[0], item_seq[1])]

    # Combine user-item and item-item pairs
    edges = user_item_pairs + item_item_pairs
    

    # Separate the edges into two lists: source and target
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    
    return edge_index

train_graph = build_graph(train_sequences)


## Padding Sequences

To ensure consistent input sizes, sequences are padded to a fixed length. This is crucial for batching and feeding sequences into the GNN model.


In [6]:

import numpy as np


def pad_sequences(seqs, max_length):
    """
    Pads sequences to the same length for batching.
    Args:
        seqs (DataFrame): Sequences of product IDs.
        max_length (int): Maximum length to pad sequences to.
    Returns:
        DataFrame: Sequences padded to the maximum length.
    """
    # Convert product_id to a list of tuples if it's not already
    seqs['itemID'] = seqs['itemID'].apply(lambda x: x if isinstance(x, tuple) else tuple(x))

    # Create a new DataFrame for padded sequences
    padded_seqs = np.zeros((len(seqs), max_length), dtype=int)

    # Fill the padded_seqs with the original product_id values
    for idx, seq in enumerate(seqs['itemID']):
        padded_seqs[idx, :len(seq)] = seq[:max_length]

    # Update the original DataFrame with padded sequences
    seqs['itemID'] = list(map(tuple, padded_seqs))

    return seqs

class SequenceDataset(torch.utils.data.Dataset):
    def __init__(self, sequences,max_length=10):
        self.sequences = pad_sequences(sequences,max_length)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        product_ids = self.sequences.iloc[idx]['itemID']
        if isinstance(product_ids, str):
            product_ids = eval(product_ids)  # Convert string representation of list to list
        return torch.tensor(product_ids, dtype=torch.long)

    
train_dataset = SequenceDataset(train_sequences)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)


In [7]:
import torch.optim as optim
num_users = train_data['userID'].nunique()
num_items = train_data['itemID'].nunique()
embedding_dim = 128

model = GNNRecommender(num_users, num_items, embedding_dim)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-5)

from tqdm import tqdm
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc='Training', leave=False):
        optimizer.zero_grad()
        out = model(train_graph, batch)
        batch = batch.view(-1)  # Flatten target: (batch_size, seq_length) -> (batch_size * seq_length)
        out = out.view(-1, num_items)  # Flatten output: (batch_size, seq_length, num_items) -> (batch_size * seq_length, num_items)
        loss = criterion(out, batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch: {epoch+1}, Train Loss: {total_loss/len(train_loader)}')

    

Epoch: 1, Train Loss: 3.502851342929984


Epoch: 2, Train Loss: 0.8693977109666141


Epoch: 3, Train Loss: 0.29479610103200066


Epoch: 4, Train Loss: 0.11140844526718248


Epoch: 5, Train Loss: 0.047410910509807884


In [8]:
# Save the model's state dictionary
torch.save(model.state_dict(), 'model_state_lstm_2.pth')

# Optionally, save the optimizer's state dictionary
torch.save(optimizer.state_dict(), 'optimizer_state_lstm_2.pth')

In [20]:
from sklearn.preprocessing import LabelEncoder
test_data = pd.read_parquet(r"/Users/vishnusai/Downloads/archive-4/test.parquet")
test_data = test_data.sample(test_data.shape[0]//2,random_state=0)
test_data.drop_duplicates(inplace=True)
test_data = test_data[test_data['product_id']!=0]

test_data = test_data[["user_id", "product_id", "timestamp"]]
test_data.rename(columns = {'user_id':'userID','product_id':'itemID','timestamp':'time'}, inplace = True)
test_data = filter_k_core(test_data, 10)

label_encoder3 = LabelEncoder()


test_data['userID'] = label_encoder3.fit_transform(test_data['userID'])
test_data['itemID'] = label_encoder3.fit_transform(test_data['itemID'])
test_sequences = test_data.groupby('userID')['itemID'].apply(list).reset_index()
test_sequences['sequences_length'] = test_sequences['itemID'].apply(len)
average_length = test_sequences['sequences_length'].mean()
print(f"The average length of the sequences is: {average_length}")

The average length of the sequences is: 15.091166077738515


In [21]:
dataset = {}
dataset["user_train"]=dict(zip(train_sequences['userID'], train_sequences['itemID']))
dataset["user_test"]=dict(zip(test_sequences["userID"],test_sequences["itemID"]))
dataset["usernum"]=test_data["userID"].nunique()
dataset["itemnum"]=test_data["itemID"].nunique()
dataset["itemnum"],dataset["usernum"]


(1216, 4245)

In [22]:
test_graph = build_graph(test_sequences)
test_dataset = SequenceDataset(test_sequences)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)
print(len(test_loader))

34


## Model Evaluation: HR@10 and NDCG@10
This section evaluates the model performance using the HR@10 and NDCG@10 metrics. These metrics are commonly used for ranking tasks, where the goal is to rank items based on their relevance to a given query.

HR@10 (Hit Rate at 10) measures the proportion of relevant items that appear within the top 10 ranked items. It is calculated as:

HR@10 = (Number of relevant items in top 10) / (Total number of relevant items)

NDCG@10 (Normalized Discounted Cumulative Gain at 10) measures the relevance of the ranked items and penalizes items that are ranked too low. It is calculated as:

NDCG@10 = (DCG@10) / (IDCG@10)

where DCG@10 is the Discounted Cumulative Gain at 10, and IDCG@10 is the Ideal Discounted Cumulative Gain at 10.

In [23]:
import random
def evaluate(model, dataset, edge_index):
    usernum = dataset["usernum"]
    itemnum = dataset["itemnum"]
    train = dataset["user_train"]
    test = dataset["user_test"]
    
    NDCG = 0.0
    HT = 0.0
    valid_user = 0.0
    
    if usernum > 10000:
        users = random.sample(range(usernum), 10000)
    else:
        users = range(usernum)
    er = 0
    for u in tqdm(users, ncols=70, leave=False, unit="b"):
    
        if len(train[u]) < 1 or len(test[u]) < 1:
            continue
    
        seq = torch.zeros(10, dtype=torch.long)
        idx = 10 - 1
        idx -= 1
        for i in reversed(train[u]):
            seq[idx] = i
            idx -= 1
            if idx == -1:
                break
        rated = set(train[u])
        rated.add(0)
        test_item = test[u][0]
        item_idx = [test_item]
        for _ in range(100):
            t = np.random.randint(1, itemnum + 1)
            while t in rated:
                t = np.random.randint(1, itemnum + 1)
            item_idx.append(t)
    
        seq = seq.unsqueeze(0)  # Add batch dimension
        item_idx_tensor = torch.tensor(item_idx).long()  # Convert to tensor
        
        with torch.no_grad():
            predictions = model.forward(edge_index, seq)
            predictions = predictions.squeeze(0)  # Remove batch dimension

            # Extract the last sequence step's prediction (most recent interaction)
            predictions = predictions[-1, :]  # Shape: [num_items]
            
            # Gather predictions for the specified item indices
            predictions = predictions[item_idx_tensor]  # Shape: [num_candidates]
            

    
        # Inverse to get descending sort
        predictions = -1.0 * predictions
        rank = predictions.argsort().argsort()
        # print(rank,predictions)
        # rank for the test item (the first item in item_idx)
        test_item_rank = rank[0].item()

        valid_user += 1
    
        if test_item_rank < 10:
            NDCG += 1 / np.log2(test_item_rank + 2)
            HT += 1
    
    return NDCG / valid_user, HT / valid_user

In [27]:
from recommenders.utils.timer import Timer
with Timer() as test_time:
    t_test = evaluate(model,dataset,test_graph)
print(t_test)

(0.1351980332556923, 0.20954063604240283)


## Conclusion

This notebook demonstrated how to implement a GNN-based sequential recommendation system. After training, we evaluated the model's performance using the ROC-AUC score. In future iterations, hyperparameter tuning and additional metrics could be incorporated to further improve the model's performance.
